<a href="https://colab.research.google.com/github/konsin1988/machine_learning_in_python/blob/master/CatBoostRegressor_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
mov = pd.read_csv('/content/drive/MyDrive/datasets/movie_dataset.csv')
mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [51]:
mov = mov[['genres', 'director', 'budget', 'release_date', 'popularity']]
mov.head()

,genres,director,budget,release_date,popularity
0,Action Adventure Fantasy Science Fiction,James Cameron,237000000,2009-12-10,150.437577
1,Adventure Fantasy Action,Gore Verbinski,300000000,2007-05-19,139.082615
2,Action Adventure Crime,Sam Mendes,245000000,2015-10-26,107.376788
3,Action Crime Drama Thriller,Christopher Nolan,250000000,2012-07-16,112.312950
4,Action Adventure Science Fiction,Andrew Stanton,260000000,2012-03-07,43.926995


In [52]:
mov = (
    mov
    .assign(release_year = lambda x: x['release_date'].astype('datetime64[ns]').dt.year)
    .drop(columns = ['release_date'])
)
mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        4775 non-null   object 
 1   director      4773 non-null   object 
 2   budget        4803 non-null   int64  
 3   popularity    4803 non-null   float64
 4   release_year  4802 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 187.7+ KB


In [53]:
mov.isna().sum()

,0
genres,28
director,30
budget,0
popularity,0
release_year,1


In [54]:
mov = mov.dropna(axis = 0, how = 'any')

In [55]:
X, y = mov.drop(columns = ['popularity']), mov['popularity']

In [56]:
X = (
    X
    .assign(genres = lambda x: x['genres'].astype('category'),
            director = lambda x: x['director'].astype('category'))
)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [58]:
model = CatBoostRegressor(iterations = 500, learning_rate=0.1, depth = 6, verbose = 0)

In [59]:
model.fit(X_train, y_train, cat_features=['genres', 'director'])

In [60]:
y_predict = model.predict(X_test)

In [62]:
root_mean_squared_error(y_predict, y_test)

25.69811006851322